In [1]:
import os
import math
import pandas as pd
from random import randint
from datetime import datetime
from scipy.special import softmax
import numpy as np
from numpy import argmax

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from tensorflow.keras import layers

import tensorflow_datasets as tfds

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from affectnet import AffectNet
from dataset_builder import AffectNetDatasetBuilder
from data_pipeline import random_flip_left_right, random_rotate, random_translate, random_hue, random_contrast, random_gamma, random_jpeg_quality

import wandb
from wandb.keras import WandbCallback

# from callbacks import TelegramNotifier

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

/home/luay/.conda/envs/tf/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
import sys
print(sys.executable)
import tensorflow
print(tensorflow.__version__)
import keras
print(keras.__version__)

/home/luay/.conda/envs/tf/bin/python
2.7.0
2.7.0


In [3]:
# sweep_config = {
#     "name" : "baseline-sweep",
#     "method" : "bayes",
#     "metric": {
#         "name": "val_accuracy",
#         "goal": "maximize"  
#     },
#     "parameters" : {
#         "learning_rate" :{
#             "min": math.log(1e-5),
#             "max": math.log(1e-4),
#             "distribution": "log_uniform"
#         },
#         "batch_size": {
#             "min": math.log(32),
#             "max": math.log(96),
#             "distribution": "q_log_uniform"
#         },
#         "base_model_output_layer_dropout": {
#             "min": 0.0,
#             "max": 0.8,
#             "distribution": "uniform"
#         },
#         "num_middle_layers": {
#             "min": 0,
#             "max": 2,
#             "distribution": "int_uniform"
#         },
#         "middle_layer_0": {
#             "min": math.log(128),
#             "max": math.log(1024),
#             "distribution": "log_uniform"
#         },
#         "middle_layer_0_dropout": {
#             "min": 0.0,
#             "max": 0.8,
#             "distribution": "uniform"
#         },
#         "middle_layer_1": {
#             "min": math.log(128),
#             "max": math.log(1024),
#             "distribution": "log_uniform"
#         },
#         "middle_layer_1_dropout": {
#             "min": 0.0,
#             "max": 0.8,
#             "distribution": "uniform"
#         },
#         "random_flip_left_right": {
#             "value": True,
#             "distribution": "constant"
#         },
#         "random_translate": {
#             "value": True,
#             "distribution": "constant"
#         },
#         "random_hue": {
#             "value": True,
#             "distribution": "constant"
#         },
#         "random_contrast": {
#             "value": True,
#             "distribution": "constant"
#         },
#         "random_gamma": {
#             "value": True,
#             "distribution": "constant"
#         },
#         "random_jpeg_quality": {
#             "value": True,
#             "distribution": "constant"
#         },
#         "random_rotate": {
#             "value": True,
#             "distribution": "constant"
#         }
#     }
# }
# sweep_id = wandb.sweep(sweep_config, project="FacialEmotionClassifier")
# print(sweep_id)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: immqshxd
Sweep URL: https://wandb.ai/luaygz/FacialEmotionClassifier/sweeps/immqshxd
immqshxd


In [4]:
SAVE_ROOT_DIR = "models"

In [5]:
class_names = ["Neutral", "Happiness", "Sadness", "Surprise", "Fear", "Disgust", "Anger", "Contempt"]

In [10]:
train_df = pd.read_csv("dataset/training.csv")
val_df = pd.read_csv("dataset/validation.csv")

train_df["expression"].value_counts()

1    134415
0     74874
2     25459
6     24882
3     14090
4      6378
5      3803
7      3750
Name: expression, dtype: int64

In [7]:
# train_df = train_df.iloc[:1250]
# train_df["expression"].value_counts()

In [13]:
def get_dataset(config, train_df, val_df, augmentations):
    dataset_builder = AffectNetDatasetBuilder(train_df, val_df)

    train_dataset, val_dataset = dataset_builder.build(batch_size=config.batch_size,
                                                       augmentations=augmentations,
                                                       shuffle=True,
                                                       seed=0)
    class_weights = dataset_builder.get_class_weights()
    
    return train_dataset, val_dataset, class_weights

In [14]:
def get_augmentations(config):
    aug = []
    if config.random_flip_left_right:
        aug.append(random_flip_left_right)
    if config.random_translate:
        aug.append(random_translate)
    if config.random_hue:
        aug.append(random_hue)
    if config.random_contrast:
        aug.append(random_contrast)
    if config.random_gamma:
        aug.append(random_gamma)
    if config.random_jpeg_quality:
        aug.append(random_jpeg_quality)
    if config.random_rotate:
        aug.append(random_rotate)
    return aug

In [15]:
def construct_model(config, class_weights=None, callbacks=None):
    base_model = ResNet50(weights="imagenet", include_top=False)
    
    if config.num_middle_layers == 0:
        middle_layer = None
    elif config.num_middle_layers == 1:
        middle_layer = Sequential([
                                   layers.Dense(config.middle_layer_0, activation="relu"),
                                   layers.Dropout(config.middle_layer_0_dropout),
                                  ])
    elif config.num_middle_layers == 2:
        middle_layer = Sequential([
                                   layers.Dense(config.middle_layer_0_dropout, activation="relu"),
                                   layers.Dropout(config.middle_layer_0_dropout),
                                   layers.Dense(config.middle_layer_1, activation="relu"),
                                   layers.Dropout(config.middle_layer_1_dropout),
                                  ])
    affectnet = AffectNet(base_model=base_model,
                      base_model_output_dropout=config.base_model_output_layer_dropout,
                      middle_layer=middle_layer,
                      preprocess_input=preprocess_input,
                      class_weights=class_weights,
                      # learning_rate=config.learning_rate,
                      learning_rate=5e-5,
                      callbacks=callbacks)
    affectnet.unfreeze(0) # Freeze all
    
    return affectnet

In [17]:
def get_callbacks():
    checkpoint_dir = datetime.today().strftime("%Y-%m-%d-%H:%M:%S") + "-" + wandb.run.name
    full_save_path = os.path.join(SAVE_ROOT_DIR, checkpoint_dir)
    os.makedirs(full_save_path)
    run_id = 0

    checkpoint_format = str(run_id) + "_epoch_{epoch:03d}.val_loss_{val_loss:.4f}"
    filepath = os.path.join(full_save_path, checkpoint_format)

    checkpoint_callback = ModelCheckpoint(filepath=filepath,
                                          save_weights_only=False,
                                          save_format="tf",
                                          monitor="val_accuracy",
                                          save_freq="epoch")

    callbacks = [checkpoint_callback]
    # callbacks.append(TelegramNotifier(checkpoint_dir, run_id))
    callbacks.append(WandbCallback(save_model=False))
    
    return callbacks

In [18]:
def train():
    with wandb.init() as run:
        config = wandb.config
        
        augmentations = get_augmentations(config)
        train_dataset, val_dataset, class_weights = get_dataset(config, train_df, val_df, augmentations)
        callbacks = get_callbacks()
        affectnet = construct_model(config, class_weights=class_weights, callbacks=callbacks)
        
        print(affectnet.model.summary())
        if affectnet.middle_layer:
            print(affectnet.middle_layer.summary())
        print(config)
        
        y_true = []
        for batch in val_dataset:
            y_true.extend(batch[1].numpy().tolist())
        
        for epoch in range(3):
            affectnet.train(train_dataset=train_dataset, val_dataset=val_dataset, epochs=1)
            pred = affectnet.model.predict(val_dataset)
            pred = pred["output_expression"]
            pred = softmax(pred)
            pred = argmax(pred, axis=1)
            wandb.log({"conf_mat": wandb.plot.confusion_matrix(probs=None,
                                                               y_true=y_true, preds=pred,
                                                               class_names=class_names)})

In [19]:
count = 1
wandb.agent("immqshxd", function=train, count=count, project="FacialEmotionClassifier")

wandb: Agent Starting Run: aso014zy with config:
wandb: 	base_model_output_layer_dropout: 0.6986851781272403
wandb: 	batch_size: 65
wandb: 	learning_rate: 7.767164497867954e-05
wandb: 	middle_layer_0: 645.9481648134051
wandb: 	middle_layer_0_dropout: 0.7029953430271116
wandb: 	middle_layer_1: 683.4530064190985
wandb: 	middle_layer_1_dropout: 0.1167755431070801
wandb: 	num_middle_layers: 1
wandb: 	random_contrast: True
wandb: 	random_flip_left_right: True
wandb: 	random_gamma: True
wandb: 	random_hue: True
wandb: 	random_jpeg_quality: True
wandb: 	random_rotate: True
wandb: 	random_translate: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luaygz (use `wandb login --relogin` to force relogin)


2022-02-18 08:43:03.468189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 08:43:03.600536: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 08:43:03.600855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 08:43:03.601931: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Loaded 287651 training images, and 4000 validation images.
Model: "affect_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d (G  multiple                 0         
 lobalAveragePooling2D)                                          
                                                                 
 sequential (Sequential)     (None, 645)               1321605   
                                                                 
 output_layer (OutputLayer)  multiple                  5168      
                                                                 
Total params: 24,914,485
Trainable params: 1,326,773
Non-trainable params: 23,587,712
_________________________________________________________________
None
Model: "sequential"
____________

2022-02-18 08:43:14.450741: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-02-18 08:43:15.782951: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-18 08:43:15.783769: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-18 08:43:15.783802: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-02-18 08:43:15.784587: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-18 08:43:15.784648: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-02-18 08:43:17.667761: W tensorflow/core/common_runtime/bf

 231/4426 [>.............................] - ETA: 29:06 - loss: 3.2009 - accuracy: 0.1402

wandb: Ctrl + C detected. Stopping sweep.
Process wandb_internal:
Traceback (most recent call last):
  File "/home/luay/.conda/envs/tf/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/luay/.conda/envs/tf/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/luay/.conda/envs/tf/lib/python3.9/site-packages/wandb/sdk/internal/internal.py", line 159, in wandb_internal
    thread.join()
  File "/home/luay/.conda/envs/tf/lib/python3.9/threading.py", line 1053, in join
    self._wait_for_tstate_lock()
  File "/home/luay/.conda/envs/tf/lib/python3.9/threading.py", line 1069, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
